soliton existence boundry:
$${\zeta_0}_{max} = \frac{1}{8} \pi^2 f^2$$
$$\psi = \frac{64}{\pi^4 f^3} - i\frac{8}{\pi^2f} + \frac{1}{2} \pi f \ \mathrm{sech}{(\sqrt{\frac{\pi^2 f^2}{8 d_2}} \phi)}$$
$$\widetilde{\psi}(m) = \frac{64}{\pi^4 f^3} - i\frac{8}{\pi^2f} + \sqrt{d_2 / 2} \ \mathrm{sech}(\sqrt{2d_2}m/f)$$
$$P_{out} = (\sqrt{P_{in}} - \sqrt{\frac{\kappa_{ext}\kappa\hbar\omega_0}{2g}} \psi)^2$$


In [ ]:
# Modules import
import numpy as np
import sys
import time
from numba import jit, objmode
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from IPython import display

In [ ]:
# Parameters
mode_number = 2**16
# d_2 = 0.001
# Q_0 = 30e6
# Q_e = 5e6
# FSR = 500e9
P_in = 500e-3

list_d_2 = np.linspace(0.1, 1, 1)
list_Q_0 = np.linspace(30e6, 40e6, 1)
list_Q_e = np.linspace(0.4e6, 0.5e6, 10)
list_FSR = np.linspace(50e9, 1000e9, 1)


In [ ]:
# Constants
c_0 = 3e8
h_bar = 6.626e-34 / (2 * np.pi)
lambda_0 = 1550e-9
n_0 = 2.00
n_2 = 22e-20
thick = 0.8e-6
width = 1.8e-6

f_0 = c_0 / lambda_0
omega_0 = 2 * np.pi * f_0


In [ ]:
# Calculation
xs = np.arange(-mode_number / 2, mode_number / 2, dtype=np.complex128)
xs_freq = np.arange(-mode_number / 2, mode_number / 2, dtype=np.complex128)


def output(d_2, Q_0, Q_e, FSR, P_in):
    Q = 1 / (1 / Q_0 + 1 / Q_e)
    kappa_0 = omega_0 / Q_0
    kappa_e = omega_0 / Q_e
    kappa = kappa_0 + kappa_e
    eta = kappa_e / kappa
    V_eff = c_0 / n_0 / FSR * width * thick
    g = h_bar * omega_0**2 * c_0 * n_2 / n_0**2 / V_eff
    P_th = (h_bar * omega_0 * kappa**2) / (8 * eta * g)
    f = np.sqrt(P_in / P_th)

    A = 64/(np.pi**4 * f**3) - 1j*8/(np.pi**2 * f) + 0.5 * np.pi * f * 1 / np.cosh(np.sqrt((np.pi**2 * f**2) / (8 * d_2)) * xs / mode_number * 2*np.pi)
    # A_freq_fft = np.fft.fftshift(np.fft.fft(A)) / mode_number
    A_freq = np.sqrt(0.5*d_2) / np.cosh(xs_freq * np.sqrt(2*d_2) / f)
    A_freq[mode_number//2] += 64/(np.pi**4 * f**3) - 1j*8/(np.pi**2 * f)

    E_out = np.sqrt(P_in) - np.sqrt(kappa_e) * np.sqrt(kappa/2/g) * A * np.sqrt(h_bar * omega_0)
    freq_out = -np.sqrt(kappa_e) * np.sqrt(kappa/2/g) * A_freq * np.sqrt(h_bar * omega_0)
    freq_out[mode_number//2] += np.sqrt(P_in)
    return freq_out

# plt.figure()
# plt.title("time domain")
# plt.plot(xs[::mode_number//2**8], np.abs(A)[::mode_number//2**8])
# plt.show()

# plt.figure()
# plt.title("frequency domain")
# # plt.plot(xs[mode_number//2 - 2**7:mode_number//2 + 2**7:1], np.abs(A_freq_fft)[mode_number//2 - 2**7:mode_number//2 + 2**7:1], label='FFT')
# plt.plot(xs_freq[mode_number//2 - 2**7:mode_number//2 + 2**7:1], np.abs(A_freq)[mode_number//2 - 2**7:mode_number//2 + 2**7:1], label='Analytical')
# plt.legend()
# plt.show()

# plt.figure()
# plt.plot(np.abs(E_out)**2, label="output", alpha=0.7)
# plt.legend()
# plt.title("Output power waveform")
# plt.xlabel("intra-cavity slice")
# plt.ylabel("Power")
# plt.ylim(0, 1.2 * np.max(np.abs(E_out)**2))
# plt.show()


plt.figure()
for d_2 in list_d_2:
    for Q_0 in list_Q_0:
        for Q_e in list_Q_e:
            for FSR in list_FSR:
                plt.plot(xs_freq[mode_number//2 - 2**7:mode_number//2 + 2**7:1], np.abs(output(d_2, Q_0, Q_e, FSR, P_in)**2)[mode_number//2 - 2**7:mode_number//2 + 2**7:1], label=f"{d_2}, {"%.1f" % (Q_0/10**6)}M, {"%.2f" % (Q_e/10**6)}M, {FSR/10**9}G", alpha=0.7)
plt.legend(fontsize=7)
plt.title("Output spectrum")
plt.xlabel("Frequency")
plt.ylabel("Power")
plt.yscale("log")
plt.show()


In [ ]:
def ratio(d_2, Q_0, Q_e, FSR):
    spectrum = np.abs(output(d_2, Q_0, Q_e, FSR, P_in)**2)
    ratio = 1 - spectrum[mode_number//2] / np.sum(spectrum)
    return ratio

for d_2 in list_d_2:
    for Q_0 in list_Q_0:
        for Q_e in list_Q_e:
            for FSR in list_FSR:
                print(f"d_2 = {d_2}, Q_0 = {"%.2f" % (Q_0/10**6)}M, Q_e = {"%.2f" % (Q_e/10**6)}M, FSR = {FSR/10**9}G, ratio = {"%.4f" % (ratio(d_2, Q_0, Q_e, FSR))}")
